In [1]:
import keras
import csv
import os
import h5py
import itertools
import scipy.io
import numpy as np
import tensorflow as tf
from keras.layers import *
from keras.applications.inception_v3 import InceptionV3
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.xception import Xception
from keras.applications.densenet import DenseNet121
from keras import backend as k
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model

Using TensorFlow backend.


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
ROOT_DIR = os.path.normpath(os.path.join(os.path.dirname(os.path.realpath('__file__'))))
SOURCE_DIR = os.path.abspath('/data/aumkar/train')
TARGET_DIR = os.path.abspath('/data/aumkar/validation')
DATA_LOAD_DIR = os.path.abspath('/data/aumkar/data_load')

In [4]:
img_width, img_height = 216, 384
batch_size = 8

In [5]:
source_gen = ImageDataGenerator(rescale=1. / 255, zoom_range=0.2, horizontal_flip = True, vertical_flip = True)

In [6]:
source_generator = source_gen.flow_from_directory(SOURCE_DIR, target_size=(img_width, img_height), 
                                                batch_size=batch_size, class_mode='categorical')

Found 152397 images belonging to 13 classes.


In [7]:
target_gen = ImageDataGenerator(rescale=1. / 255, zoom_range=0.2, horizontal_flip = True, vertical_flip = True)

In [8]:
target_generator = target_gen.flow_from_directory(TARGET_DIR, target_size=(img_width, img_height), 
                                                batch_size=batch_size, class_mode='categorical')

Found 55388 images belonging to 13 classes.


In [9]:
y_source = source_generator.classes
y_target = target_generator.classes

In [10]:
input_ = Input(shape=(img_width, img_height, 3))

In [11]:
def model_create(transfer_model):
    if transfer_model == 'InceptionV3':
        with tf.device('/gpu:1'):
            model_ = InceptionV3(input_tensor = input_, weights = 'imagenet', include_top = False)
        return Model(inputs = model_.input, outputs = model_.output)
    elif transfer_model == 'VGG16':
        with tf.device('/gpu:1'):
            model_ = VGG16(input_tensor = input_, weights = 'imagenet', include_top = False)
        return Model(inputs = model_.input, outputs = model_.output)
    elif transfer_model == 'VGG19':
        with tf.device('/gpu:1'):
            model_ = VGG19(input_tensor = input_, weights = 'imagenet', include_top = False)
        return Model(inputs = model_.input, outputs = model_.output)
    elif transfer_model == 'Xception':
        with tf.device('/gpu:1'):
            model_ = Xception(input_tensor = input_, weights = 'imagenet', include_top = False)
        return Model(inputs = model_.input, outputs = model_.output)
    elif transfer_model == 'DenseNet121':
        with tf.device('/gpu:1'):
            model_ = DenseNet121(input_tensor = input_, weights = 'imagenet', include_top = False)
        return Model(inputs = model_.input, outputs = model_.output)

In [12]:
def feature_extract(gen, model):
    return model.predict_generator(gen, steps = batch_size)

In [13]:
def save_features(file, feature):
    print('Saving {}'.format(file))
    scipy.io.savemat(file, {'x': feature[0], 'y': feature[1]}) 

In [14]:
for model in ['InceptionV3', 'VGG16', 'VGG19', 'Xception', 'DenseNet121']:
    
    filename = '{}/{}.mat'.format(DATA_LOAD_DIR, model)
    filename_target = '{}/{}_target.mat'.format(DATA_LOAD_DIR, model)
    transfer_model = model_create(model)
    print('Extracting features from source data for model {}'.format(model))
    features = feature_extract(source_generator, transfer_model)
    print('Extracting features from target data for model {}'.format(model))
    features_target = feature_extract(target_generator, transfer_model)
    
    for feature in [(features, y_source)]:
        save_features(filename, feature)
        
    for feature in [(features_target, y_target)]:
        save_features(filename_target, feature)

Extracting features from source data for model InceptionV3
Extracting features from target data for model InceptionV3
Saving /data/aumkar/data_load/InceptionV3.mat
Saving /data/aumkar/data_load/InceptionV3_target.mat
Extracting features from source data for model VGG16
Extracting features from target data for model VGG16
Saving /data/aumkar/data_load/VGG16.mat
Saving /data/aumkar/data_load/VGG16_target.mat
Extracting features from source data for model VGG19
Extracting features from target data for model VGG19
Saving /data/aumkar/data_load/VGG19.mat
Saving /data/aumkar/data_load/VGG19_target.mat
Extracting features from source data for model Xception
Extracting features from target data for model Xception
Saving /data/aumkar/data_load/Xception.mat
Saving /data/aumkar/data_load/Xception_target.mat
Extracting features from source data for model DenseNet121
Extracting features from target data for model DenseNet121
Saving /data/aumkar/data_load/DenseNet121.mat
Saving /data/aumkar/data_loa